In [2]:
import re, string
from collections import Counter
import numpy as np
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

GENERATED_DATA = './generated_datasets/'
DS_PATH = './hwk3_datasets/'
DS_TYPES = ['train', 'valid', 'test']
DS = ['yelp-', 'IMDB-']
AVERAGE = 'micro'
NUM_FEATURES = 10000

# Extract the top features from the dataset and write them to a file
def extract_features(dataset):
    file = DS_PATH + dataset + DS_TYPES[0] + '.txt'
    reviews = []

    # find top features and write to file
    with open(file, 'r', encoding="utf-8") as f:
        for l in f.readlines():
            sp = l.split('\t')
            reviews.append(re.compile('[^\w\s]').sub('', sp[0].strip()).lower())
    words = Counter([word for line in reviews for word in line.split()]).most_common(NUM_FEATURES)
    dictionary = {}
    writer = open(GENERATED_DATA + dataset.split('-')[0] + '-vocab.txt', 'w')
    for i in range(NUM_FEATURES):
        word = words[i][0]
        dictionary[word] = i + 1
        text = ("{}\t{}\t{}\n".format(word, i + 1, words[i][1]))
        writer.write(text)

    # create feature vectors and write to file
    for type in DS_TYPES:
        file = DS_PATH + dataset + type + '.txt'
        translator = str.maketrans(" ", " ", string.punctuation)
        with open(file, 'r', encoding="utf-8") as f:
            text = f.read()
        examples = text.lower().replace('\t', ' ').replace('<br /><br />', ' ').translate(translator).split("\n")[:-1]
        output = [i[-1] for i in examples]
        writer = open(GENERATED_DATA + dataset.split('-')[0] + '-' + type.split('.')[0] + '.txt', 'w')
        for i in range(len(examples)):
            text = ''
            for word in examples[i].split(' ')[:-1]:
                if word in dictionary.keys():
                    text = '{} {}'.format(text, dictionary[word])
            if len(text) == 0: text = ' '
            text = "{}\t{}\n".format(text, output[i])
            writer.write(text[1:])

    return dictionary

# Coverts a dataset to bag of words representation
def to_bow(features, set):
    binary = {}
    freq = {}
    for type in DS_TYPES:
        # read text from file
        file = DS_PATH + set + type + '.txt'
        translator = str.maketrans(" ", " ", string.punctuation)
        with open(file, 'r', encoding="utf-8") as f:
            text = f.read()
        text = list(filter(None, text.lower().replace('\t', ' ').replace('<br /><br />', ' ').translate(translator).split('\n')))
        input = [line[:-1] for line in text]
        output = np.array([int(line[-1]) for line in text])
        # use vectorizer object create binary and frequency bag of words
        vectorizer = CountVectorizer(vocabulary=features.keys())
        vectors = np.asarray(vectorizer.fit_transform(input).todense())
        freq[type] = [sparse.csr_matrix(normalize(vectors)), output]
        vectors[vectors > 1] = 1
        binary[type] = [sparse.csr_matrix(vectors), output]
    return binary, freq

''' Part 1 - Cleaning and Vectorizing Data \n '''
print('Part 1 - Cleaning and Vectorizing Data \n')

# Clean and vectorize yelp data
features = extract_features(DS[0])
yelp_binary, yelp_freq = to_bow(features, DS[0])

# Clean and vectorize IMDB data
features = extract_features(DS[1])
imdb_binary, imdb_freq = to_bow(features, DS[1])

print('Dataset cleaned and vectorized. Results can be found in generated folder. \n')

Part 1 - Cleaning and Vectorizing Data 



MemoryError: 